In [7]:
import os.path
from urllib import request
from bs4 import BeautifulSoup
import geopandas as gp
import numpy as np
import pandas as pd

In [3]:
locs_url = "https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Los_Angeles"

wikibase = "https://en.wikipedia.org"

res = request.urlopen(locs_url).read()
soup = BeautifulSoup(res,'html.parser')

loc_lists = soup.find_all('ul')[1:26]

masterlist = []

for loc in loc_lists:
    masterlist += [li.a['href'] for li in loc.find_all('li')]

    
nrows = len(masterlist)
columns=['title','lon','lat','thumb_im_url','full_im_url']
wiki_df = pd.DataFrame(columns=columns,index=np.arange(nrows))

In [4]:
failcount = 0
for ind,page in enumerate(masterlist):

    tmpurl = wikibase + page
    tmpres = request.urlopen(tmpurl).read()
    tmpsoup = BeautifulSoup(tmpres,'html.parser')
    #print(page)
    if tmpsoup:
        try:
            lat = tmpsoup.find(class_=['latitude']).text
            lon = tmpsoup.find(class_=['longitude']).text
        except AttributeError:
            #print('failed at coords... skipping')
            failcount += 1
            continue
            
        cardobj = tmpsoup.find(class_=['infobox'])
        try:
            imobj = cardobj.find('a',class_='image')
            imlink = imobj.img['src']
            fullimglink = os.path.dirname(imlink)
        except AttributeError:
            #print('FUCKING CARDBOX!!!!') #TODO: can look for another element with pic later...
            imlink = None
            fullimglink = None
            
        name = page.split('/')[-1]
        wiki_df.iloc[ind] = [name,lon,lat,imlink,fullimglink]
    else:
        print('tmpsoup be empty')
        failcount +=1

In [6]:
pwd

'/Users/karlen/Work/Insight/BikesploreLA'

In [5]:
wiki_df.to_csv('wiki_sights_updated.csv')

TRAFFIC VOLUME INFO OH SNAPPPPPPY!

In [2]:
import pysal as ps
import pandas as pd 

/Users/karlen/anaconda3/envs/insight/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [7]:
for year in range(2015,2017):

    dbf = ps.lib.io.open('../data/raw/traffic_volume/AADT{}.dbf'.format(year))
    d = {col: dbf.by_col(col) for col in dbf.header}
    traffic_vol_df = pd.DataFrame(d)
    traffic_vol_df.to_csv('../data/raw/traffic_volume/AADT{}.csv'.format(year))

In [159]:
traffic_vol_df.to_csv('../data/traffic_volume/AADT2017.csv')